In [21]:
# 导入必要的库
import sys
import os

# 添加项目根目录到Python路径
sys.path.insert(0, os.path.abspath('../../'))

# 导入数据库相关模块
from table.order import order  # 导入新表的ORM
from sql.database import get_db_session, get_db
import pandas as pd

In [22]:
# 连接数据库并查询order表的数据
with get_db_session() as db:
    records = db.query(order).all()

    # 获取列名
    if records:
        column_names = records[0].__table__.columns.keys()
        data = [{column: getattr(record, column) for column in column_names} for record in records]
        df = pd.DataFrame(data)
        print(f"从order表中查询到 {len(df)} 条记录")
    else:
        df = pd.DataFrame()
        print("order表中没有记录")


2025-11-03 20:41:41,306 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-03 20:41:41,307 INFO sqlalchemy.engine.Engine SELECT "order".id AS order_id, "order"."ORDER_NO" AS "order_ORDER_NO", "order"."ORDER_MONTH" AS "order_ORDER_MONTH", "order"."ORDER_WT" AS "order_ORDER_WT", "order"."DELIVERY_DATE" AS "order_DELIVERY_DATE" 
FROM "order"
2025-11-03 20:41:41,307 INFO sqlalchemy.engine.Engine [cached since 2055s ago] ()
从order表中查询到 670 条记录
2025-11-03 20:41:41,314 INFO sqlalchemy.engine.Engine ROLLBACK


In [30]:
# 拿到11月的合同数据
df_sorted_by_order_month = df.sort_values(by=['ORDER_MONTH'])
df_filtered = df_sorted_by_order_month[df_sorted_by_order_month['ORDER_MONTH'] == '202411']

orders = df_filtered.head(10)

from order_plan_parameter import process_num, speed, transmission_time

orders_static_parameter = {}
# 数据格式
# {
#     "order_no": {
#         "process_num": {
#             "process_time": 12,
#             "transmission_time": 3
#         }
#     }
# }


# 获取订单号列表
order_nos = orders["ORDER_NO"].tolist()

for idx, order_no in enumerate(order_nos):
    orders_static_parameter[order_no] = {}

    # 对于每个工序
    for i in range(process_num):
        # 计算处理时间（注意需要转换为list后再索引）
        process_time = (orders["ORDER_WT"] / speed[i]).tolist()[idx]
        # 获取传输时间
        trans_time = transmission_time[i]

        # 构建嵌套字典结构
        orders_static_parameter[order_no][i] = {
            'process_time': process_time,
            'transmission_time': trans_time
        }


In [4]:
# 画图
import plotly.io as pio
pio.templates.default = "plotly_white"


def plot_schedule_gantt(scheduled_df, title="生产调度甘特图"):
    """
    使用 Plotly 绘制调度结果的甘特图，Y轴按工序显示，条形按订单号区分。

    Args:
        scheduled_df (pd.DataFrame): heuristic_scheduler 返回的调度结果DataFrame。
        static_params (dict): 静态参数，用于获取处理时间。
        title (str): 甘特图的标题。
    """

    gantt_data = []

    for idx, row in scheduled_df.iterrows():
        order_no = row['ORDER_NO']

        # 酸轧工序任务
        gantt_data.append(dict(
            Task=order_no, # 每个任务的名称是订单号
            Start=row['S_AZ'],
            Finish=row['E_AZ'],
            Resource="酸轧", # Y轴的类别
            ORDER_NO=order_no,
            Process_Time_Hours=row['PROCESS_TIME_AZ']
        ))
        # 连退工序任务
        gantt_data.append(dict(
            Task=order_no, # 每个任务的名称是订单号
            Start=row['S_LT'],
            Finish=row['E_LT'],
            Resource="连退", # Y轴的类别
            ORDER_NO=order_no,
            Process_Time_Hours=row['PROCESS_TIME_LT']
        ))

    gantt_df = pd.DataFrame(gantt_data)

    # 确保 'Start' 和 'Finish' 列是 datetime 类型
    gantt_df['Start'] = pd.to_datetime(gantt_df['Start'])
    gantt_df['Finish'] = pd.to_datetime(gantt_df['Finish'])

    fig = px.timeline(gantt_df,
                      x_start="Start",
                      x_end="Finish",
                      y="Resource",  # Y轴现在是工序名称
                      color="ORDER_NO", # 条形颜色按订单号区分
                      hover_name="ORDER_NO",
                      hover_data={
                          "Start": "|%Y-%m-%d %H:%M",
                          "Finish": "|%Y-%m-%d %H:%M",
                          "Resource": True,
                          "ORDER_NO": False, # 已经显示在hover_name中，这里不再重复
                          "Process_Time_Hours": True
                      },
                      title=title)

    # 调整Y轴的顺序，确保酸轧在连退之上
    fig.update_yaxes(categoryorder="array", categoryarray=[ "连退", "酸轧"])

    fig.update_layout(
        xaxis_title="时间",
        yaxis_title="工序",
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
        height=len(gantt_df['Resource'].unique()) * 100 + 200, # 根据工序数量调整高度
        title_x=0.5, # 标题居中
        # 调整刻度显示，避免过于密集
        xaxis=dict(
            tickformat="%Y-%m-%d\n%H:%M", # 显示年-月-日 换行 小时:分钟
        )
    )
    fig.show()

# --- 新增绘图函数：拖期情况柱状图 ---
def plot_delivery_performance(scheduled_df, title="订单交货情况 (拖期/提前)"):
    """
    绘制订单交货情况的柱状图。
    向上红色表示拖期，向下绿色表示提前交货。

    Args:
        scheduled_df (pd.DataFrame): heuristic_scheduler 返回的调度结果DataFrame。
        title (str): 图表的标题。
    """

    plot_df = scheduled_df.copy()

    # 计算实际的交货时间差：负值表示提前，正值表示拖期
    plot_df['DELIVERY_DIFF_HOURS'] = (plot_df['E_LT'] - plot_df['DELIVERY_DATE']).dt.total_seconds() / 3600

    # 根据交货情况分配颜色
    plot_df['COLOR'] = plot_df['DELIVERY_DIFF_HOURS'].apply(lambda x: 'red' if x > 0 else 'green')

    fig = px.bar(plot_df,
                 x='ORDER_NO',
                 y='DELIVERY_DIFF_HOURS',
                 color='COLOR', # 使用自定义颜色
                 color_discrete_map={'red': 'red', 'green': 'green'}, # 明确颜色映射
                 title=title,
                 labels={
                     'ORDER_NO': '订单号',
                     'DELIVERY_DIFF_HOURS': '交货时间差 (小时, 正为拖期, 负为提前)'
                 },
                 hover_data={
                     'DELIVERY_DATE': '|%Y-%m-%d %H:%M',
                     'E_LT': '|%Y-%m-%d %H:%M',
                     'COLOR': False # 不在hover信息中显示颜色分类
                 })

    fig.update_layout(
        xaxis_title="订单号",
        yaxis_title="时间差 (小时)",
        title_x=0.5,
        showlegend=False # 不显示颜色图例，因为颜色很直观
    )

    # 添加一条零线，更清晰地表示拖期和提前的界限
    fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1)

    fig.show()

# --- 新增绘图函数：合并库存时间柱状图 ---
def plot_combined_inventory_time(scheduled_df, title="订单工序后库库存时间"):
    """
    绘制订单酸轧后库和连退后库库存时间的柱状图。
    X轴为订单号，Y轴为库存时间，颜色区分不同工序后库。

    Args:
        scheduled_df (pd.DataFrame): heuristic_scheduler 返回的调度结果DataFrame。
        title (str): 图表的标题。
    """

    # 将酸轧后库和连退后库库存数据进行重塑，以便在同一个图表中显示
    inventory_data = []
    for idx, row in scheduled_df.iterrows():
        inventory_data.append({
            'ORDER_NO': row['ORDER_NO'],
            '库存类型': '酸轧后库库存',
            '库存时间 (小时)': row['I_AZ_hours']
        })
        inventory_data.append({
            'ORDER_NO': row['ORDER_NO'],
            '库存类型': '连退后库库存', # 成品库存
            '库存时间 (小时)': row['I_LT_hours']
        })

    inventory_df = pd.DataFrame(inventory_data)

    fig = px.bar(inventory_df,
                 x='ORDER_NO',
                 y='库存时间 (小时)',
                 color='库存类型', # 根据库存类型区分颜色
                 barmode='group', # 设置为分组柱状图，而不是堆叠
                 title=title,
                 labels={
                     'ORDER_NO': '订单号',
                     '库存时间 (小时)': '库存时间 (小时)'
                 },
                 color_discrete_map={
                     '酸轧后库库存': 'skyblue',
                     '连退后库库存': 'lightcoral'
                 }
                )

    fig.update_layout(
        xaxis_title="订单号",
        yaxis_title="库存时间 (小时)",
        title_x=0.5,
        legend_title="库存类型"
    )

    fig.show()

In [29]:
# 启发式求解
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

def heuristic_scheduler(orders_df, static_params, strategy='EDD', w1=1.0, w2=0.5, w3=100):
    """
    使用启发式方法调度酸轧和连退工序，生成订单生产计划。

    Args:
        orders_df (pd.DataFrame): 包含订单信息的DataFrame，列包括 id, ORDER_NO, ORDER_MONTH, ORDER_WT, DELIVERY_DATE。
        static_params (dict): 静态参数，包含每个订单在每个工序的处理时间、传输时间等。
                              格式：{"order_no": {"process_num": {"process_time": 12, "transmission_time": 3}}}
        strategy (str): 调度策略，'EDD' (最早交货期优先), 'SPT_AZ' (酸轧最短处理时间优先),
                        'SPT_LT' (连退最短处理时间优先) 或 'FCFS' (先来先服务)。

    Returns:
        pd.DataFrame: 调度结果，包含每个订单的S_AZ, E_AZ, S_LT, E_LT, L_i, I_AZ, I_LT等信息。
    """
    scheduled_orders_results = []
    # 复制订单DataFrame，避免修改原始数据
    orders = orders_df.copy()

    # 将交货日期转换为 datetime 对象
    orders['DELIVERY_DATE'] = pd.to_datetime(orders['DELIVERY_DATE'], format='%Y%m%d')
    # 根据订单月份设置计划开始日期（默认为该月月初）
    orders['PLAN_START_DATE'] = orders['ORDER_MONTH'].apply(lambda x: pd.to_datetime(str(x)[:4] + '-' + str(x)[4:] + '-01'))

    # 遍历订单，从静态参数中提取并添加工序处理时间和传输时间到订单DataFrame
    for idx, row in orders.iterrows():
        order_no = row['ORDER_NO']

        # 酸轧工序 (process_num = 0) 参数
        az_params = static_params.get(order_no, {}).get(0, {})
        orders.loc[idx, 'AZ_PROCESS_TIME_HOURS'] = az_params.get('process_time', 0)
        orders.loc[idx, 'AZ_TRANSMISSION_TIME_HOURS'] = az_params.get('transmission_time', 0) # 酸轧到连退的物流时间

        # 连退工序 (process_num = 1) 参数
        lt_params = static_params.get(order_no, {}).get(1, {})
        orders.loc[idx, 'LT_PROCESS_TIME_HOURS'] = lt_params.get('process_time', 0)
        # 连退工序后通常无外部传输，这里传输时间可能为0
        orders.loc[idx, 'LT_TRANSMISSION_TIME_HOURS'] = lt_params.get('transmission_time', 0)

    # 将处理时间和传输时间从小时转换为 timedelta 对象，便于日期时间计算
    orders['AZ_PROCESS_TIME_TD'] = orders['AZ_PROCESS_TIME_HOURS'].apply(lambda x: timedelta(hours=x))
    orders['AZ_TRANSMISSION_TIME_TD'] = orders['AZ_TRANSMISSION_TIME_HOURS'].apply(lambda x: timedelta(hours=x))
    orders['LT_PROCESS_TIME_TD'] = orders['LT_PROCESS_TIME_HOURS'].apply(lambda x: timedelta(hours=x))

    # 初始化机组可用时间，从所有订单中最早的计划开始日期开始
    machine_available_time_az = orders['PLAN_START_DATE'].min()
    machine_available_time_lt = orders['PLAN_START_DATE'].min()

    # --- 根据调度策略对订单进行排序 ---
    # 考虑库存最小
    if strategy == 'MIN_INV':
        unscheduled = orders.copy()
        scheduled_orders_results = []

        # 设定初始的“最晚可用结束时间”为所有交货日的最大值（也可以用 datetime.max）
        max_delivery = orders['DELIVERY_DATE'].max()
        lt_latest_end = max_delivery
        az_latest_end = max_delivery

        # 权重：可调整
        # w1: 酸轧后库权重，w2: 连退后库权重，w3: 拖期惩罚权重
        # 如果没有传入这些参数，可以在函数签名给默认值，例如 w1=1.0, w2=0.5, w3=100.0
        try:
            w1; w2; w3
        except NameError:
            w1, w2, w3 = 1.0, 0.5, 100.0

        while not unscheduled.empty:
            best_candidate = None
            best_score = float('inf')

            for idx, order in unscheduled.iterrows():
                # 处理时间（小时）
                az_proc_h = order['AZ_PROCESS_TIME_HOURS']
                lt_proc_h = order['LT_PROCESS_TIME_HOURS']
                az_trans_h = order['AZ_TRANSMISSION_TIME_HOURS']

                # 先尝试从交货期向前倒推 LT 完成时间（不晚于交货日，也不晚于 LT 当前最晚可用结束）
                tentative_e_lt = min(order['DELIVERY_DATE'], lt_latest_end)
                tentative_s_lt = tentative_e_lt - timedelta(hours=lt_proc_h)

                # 预期 AZ 最晚结束 = LT 开始 - 传输（且不能晚于 az_latest_end）
                tentative_e_az = min(tentative_s_lt - timedelta(hours=az_trans_h), az_latest_end)
                tentative_s_az = tentative_e_az - timedelta(hours=az_proc_h)

                # 标记是否需要从前向后推（即倒推到计划开始前，不可行）
                forced_forward = False
                if tentative_s_az < order['PLAN_START_DATE']:
                    # 倒推不可行：把 AZ 开始强制设为计划开始，然后正向推算 AZ/ LT
                    forced_forward = True
                    tentative_s_az = order['PLAN_START_DATE']
                    tentative_e_az = tentative_s_az + timedelta(hours=az_proc_h)
                    # LT 开始不能早于 e_az + trans，也不能早于 0；LT 可能需要向后推到 az_latest_end 的限制
                    tentative_s_lt = max(tentative_e_az + timedelta(hours=az_trans_h),
                                         lt_latest_end - timedelta(days=365*100))  # keep it flexible; we handle below
                    # 为了简洁，把 tentative_s_lt 至少设为 e_az + trans
                    tentative_s_lt = tentative_e_az + timedelta(hours=az_trans_h)
                    tentative_e_lt = tentative_s_lt + timedelta(hours=lt_proc_h)
                    # 如果这导致 E_LT 超过交货日，则会产生 delay
                # 计算库存与延迟
                I_AZ = max(0.0, (tentative_s_lt - (tentative_e_az + timedelta(hours=az_trans_h))).total_seconds() / 3600.0)
                I_LT = max(0.0, (order['DELIVERY_DATE'] - tentative_e_lt).total_seconds() / 3600.0)
                delay_hours = max(0.0, (tentative_e_lt - order['DELIVERY_DATE']).total_seconds() / 3600.0)

                score = w1 * I_AZ + w2 * I_LT + w3 * delay_hours

                # 可选：轻微偏好不强制前推的方案（避免推早导致连锁影响）
                if forced_forward:
                    score += 1e-6  # tiny penalty, 可去掉或调整

                # 记录最小 score 的候选
                if score < best_score:
                    best_score = score
                    best_candidate = {
                        'idx': idx,
                        'order': order,
                        'S_AZ': tentative_s_az,
                        'E_AZ': tentative_e_az,
                        'S_LT': tentative_s_lt,
                        'E_LT': tentative_e_lt,
                        'I_AZ': I_AZ,
                        'I_LT': I_LT,
                        'delay': delay_hours
                    }

            # 固定最优候选到调度表
            if best_candidate is None:
                # 不应发生，但为安全性加个保护
                break

            b = best_candidate
            order = b['order']
            scheduled_orders_results.append({
                'ORDER_NO': order['ORDER_NO'],
                'DELIVERY_DATE': order['DELIVERY_DATE'],
                'S_AZ': b['S_AZ'],
                'E_AZ': b['E_AZ'],
                'S_LT': b['S_LT'],
                'E_LT': b['E_LT'],
                'L_i_hours': b['delay'],
                'I_AZ_hours': b['I_AZ'],
                'I_LT_hours': b['I_LT'],
                'PROCESS_TIME_AZ': static_params[order['ORDER_NO']][0]['process_time'],
                'PROCESS_TIME_LT': static_params[order['ORDER_NO']][1]['process_time']
            })

            # 更新机器的“最晚可用结束时间”：因为我们向前占用了时间段，所以机器可用的最新结束应变为该任务的开始
            lt_latest_end = b['S_LT']
            az_latest_end = b['S_AZ']

            # 从未排集合中移除
            unscheduled = unscheduled.drop(b['idx'])

        # 返回按你固定顺序（这里为向后排的顺序）构造的 DataFrame
        result_df = pd.DataFrame(scheduled_orders_results)
        # 如果需要按时间或其他排序返回，可以在这里排序
        return result_df


    if strategy == 'EDD':
        # 最早交货期优先 (Earliest Due Date)
        orders = orders.sort_values(by='DELIVERY_DATE').reset_index(drop=True)
    elif strategy == 'SPT_AZ':
        # 酸轧最短处理时间优先 (Shortest Processing Time for Acid Rolling)
        orders = orders.sort_values(by='AZ_PROCESS_TIME_HOURS').reset_index(drop=True)
    elif strategy == 'SPT_LT':
        # 连退最短处理时间优先 (Shortest Processing Time for Continuous Annealing)
        orders = orders.sort_values(by='LT_PROCESS_TIME_HOURS').reset_index(drop=True)
    elif strategy == 'FCFS':
        # 先来先服务 (First Come First Served)，按订单ID排序
        orders = orders.sort_values(by='id').reset_index(drop=True)
    else:
        raise ValueError("不支持的调度策略。请选择 'EDD', 'SPT_AZ', 'SPT_LT', 'FCFS'")

    # --- 迭代调度每个订单 ---
    for idx, order in orders.iterrows():
        order_no = order['ORDER_NO']

        # 酸轧工序调度
        # 开始时间取订单最早计划开始时间和酸轧机组可用时间的最大值
        s_az = max(order['PLAN_START_DATE'], machine_available_time_az)
        # 结束时间为开始时间加上处理时间
        e_az = s_az + order['AZ_PROCESS_TIME_TD']

        # 更新酸轧机组的可用时间为当前订单的酸轧结束时间
        machine_available_time_az = e_az

        # 连退工序调度
        # 连退最早开始时间取决于：
        # 1. 酸轧工序完成并经过传输时间后的时间 (e_az + AZ_TRANSMISSION_TIME_TD)
        # 2. 连退机组当前的可用时间 (machine_available_time_lt)
        s_lt = max(e_az + order['AZ_TRANSMISSION_TIME_TD'], machine_available_time_lt)
        # 结束时间为开始时间加上处理时间
        e_lt = s_lt + order['LT_PROCESS_TIME_TD']

        # 更新连退机组的可用时间为当前订单的连退结束时间
        machine_available_time_lt = e_lt

        # --- 结果评估和记录 ---
        # 拖期计算 (L_i_hours): 如果连退完成时间晚于交货期，则计算拖期；否则为0
        delay_hours = max(0, (e_lt - order['DELIVERY_DATE']).total_seconds() / 3600)

        # 酸轧后库库存时间 (I_AZ_hours):
        # 货物从酸轧完成并传输到连退入口后，到连退实际开始处理之间等待的时间。
        # 如果等待时间为负（即连退机组早早空闲，等待酸轧完成），则库存为0。
        inventory_az_hours = max(0, (s_lt - (e_az + order['AZ_TRANSMISSION_TIME_TD'])).total_seconds() / 3600)

        # 连退后库库存时间 (I_LT_hours，成品库存):
        # 如果订单在交货期之前完成 (E_LT < DELIVERY_DATE)，则在成品库中停留一段时间，直到交货期。
        # 否则，没有提前库存，为0。
        inventory_lt_hours = max(0, (order['DELIVERY_DATE'] - e_lt).total_seconds() / 3600)

        # 将调度结果添加到列表中
        scheduled_orders_results.append({
            'ORDER_NO': order_no,
            'DELIVERY_DATE': order['DELIVERY_DATE'],
            'S_AZ': s_az, # 酸轧开始时间
            'E_AZ': e_az, # 酸轧结束时间
            'S_LT': s_lt, # 连退开始时间
            'E_LT': e_lt, # 连退结束时间
            'L_i_hours': delay_hours, # 订单拖期小时数
            'I_AZ_hours': inventory_az_hours, # 酸轧后库库存时间
            'I_LT_hours': inventory_lt_hours, # 连退后库库存时间 (成品库存)
            'PROCESS_TIME_AZ': static_params[order_no][0]['process_time'], # 酸轧工序处理时间
            'PROCESS_TIME_LT': static_params[order_no][1]['process_time']  # 连退工序处理时间
        })

    return pd.DataFrame(scheduled_orders_results)
#
# # --- 示例数据 (同上) ---
# orders_data = {
#     'id': [1, 2, 3, 4],
#     'ORDER_NO': ['GG24003300', 'GG24003301', 'GG24003302', 'GG24003303'],
#     'ORDER_MONTH': [202411, 202411, 202412, 202412],
#     'ORDER_WT': [200.0, 150.0, 300.0, 100.0],
#     'DELIVERY_DATE': [20241231, 20241220, 20250115, 20250105]
# }
# orders_df = pd.DataFrame(orders_data)
#
# orders_static_parameter = {
#     "GG24003300": {
#         0: {"process_time": 10, "transmission_time": 3},  # 酸轧工序0, 处理10h, 传输3h
#         1: {"process_time": 8, "transmission_time": 0}   # 连退工序1, 处理8h, 传输0h
#     },
#     "GG24003301": {
#         0: {"process_time": 15, "transmission_time": 3},
#         1: {"process_time": 12, "transmission_time": 0}
#     },
#     "GG24003302": {
#         0: {"process_time": 20, "transmission_time": 3},
#         1: {"process_time": 18, "transmission_time": 0}
#     },
#      "GG24003303": {
#         0: {"process_time": 5, "transmission_time": 3},
#         1: {"process_time": 6, "transmission_time": 0}
#     }
# }

# --- 运行调度器并绘图 ---
scheduled_df_edd = heuristic_scheduler(orders, orders_static_parameter, strategy='MIN_INV')
print("---调度结果 ---")
print(scheduled_df_edd)
print(f"总拖期 (小时): {scheduled_df_edd['L_i_hours'].sum():.2f}")
print(f"总酸轧后库库存时间 (小时): {scheduled_df_edd['I_AZ_hours'].sum():.2f}")
print(f"总连退后库库存时间 (小时): {scheduled_df_edd['I_LT_hours'].sum():.2f}")

plot_schedule_gantt(scheduled_df_edd, title="生产调度甘特图")
# 调用库存时间图
plot_combined_inventory_time(scheduled_df_edd, title="合同库存时间")
# 调用交货情况图
plot_delivery_performance(scheduled_df_edd, title="合同交货情况")


# 可以比较不同策略的结果来选择更优的方案

---调度结果 ---
     ORDER_NO DELIVERY_DATE                S_AZ                E_AZ  \
0  GG24003300    2024-12-31 2024-12-30 15:10:00 2024-12-30 17:40:00   
1  G024013563    2024-12-31 2024-12-29 18:20:00 2024-12-30 04:20:00   
2  G024014114    2024-12-31 2024-12-29 00:55:00 2024-12-29 12:40:00   
3  GG24003449    2024-12-31 2024-12-28 13:20:00 2024-12-28 23:20:00   
4  G024013564    2024-12-31 2024-12-28 00:00:00 2024-12-28 10:00:00   
5  GG24003314    2024-12-31 2024-12-27 10:40:00 2024-12-27 20:40:00   
6  GG24003298    2024-12-31 2024-12-27 04:10:00 2024-12-27 10:40:00   
7  GG24003445    2024-12-31 2024-12-26 23:10:00 2024-12-27 04:10:00   
8  GG24003326    2024-12-31 2024-12-26 20:40:00 2024-12-26 23:10:00   
9  G024013530    2024-12-31 2024-12-26 18:40:00 2024-12-26 20:40:00   

                 S_LT                E_LT  L_i_hours  I_AZ_hours  I_LT_hours  \
0 2024-12-30 20:40:00 2024-12-31 00:00:00        0.0    0.000000    0.000000   
1 2024-12-30 07:20:00 2024-12-30 20:40:00     